In [92]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd

In [93]:
from typing import Dict, List, Tuple, Any
import pandas as pd
import os

# Visualization Class (Completely AI Generated)

In [ ]:
class WeatherPlotter:
    """
    Interactive weather data plotter for Jupyter/Colab notebooks.

    Attributes:
    -----------
    cleaned_dfs : dict
        Dictionary of cleaned dataframes {station_name: df}
    variable_map : dict
        Dictionary mapping stations to their available variables
    """

    def __init__(
        self,
        cleaned_dfs: Dict[str, pd.DataFrame],
        variable_map: Dict[str, List[str]]
    ) -> None:
        """
        Initialize the WeatherPlotter.

        Parameters:
        -----------
        cleaned_dfs : dict
            Dictionary of cleaned dataframes {station_name: df}
        variable_map : dict
            Dictionary mapping stations to their available variables
        """
        self.cleaned_dfs: Dict[str, pd.DataFrame] = cleaned_dfs
        self.variable_map: Dict[str, List[str]] = variable_map
        self.stations: List[str] = list(cleaned_dfs.keys())
        # Create widgets
        self.station_dropdown: widgets.Dropdown = widgets.Dropdown(
            options=self.stations,
            value=self.stations[0],
            description='Station:',
            style={'description_width': 'initial'}
        )
        self.variable_dropdown: widgets.Dropdown = widgets.Dropdown(
            options=[],
            description='Variable:',
            style={'description_width': 'initial'}
        )
        self.output: widgets.Output = widgets.Output()
        # Set up observers
        self.station_dropdown.observe(self.on_station_change, names='value')
        self.variable_dropdown.observe(self.on_variable_change, names='value')

        # Initialize variables for first station
        self.update_variables()

    def update_variables(self) -> None:
        """Update available variables based on selected station."""
        selected_station: str = self.station_dropdown.value
        available_vars: List[str] = self.variable_map.get(selected_station, [])
        self.variable_dropdown.options = available_vars
        if available_vars:
            self.variable_dropdown.value = available_vars[0]

    def on_station_change(self, change: Dict[str, Any]) -> None:
        """Callback when station dropdown changes."""
        self.update_variables()
        self.plot_data()

    def on_variable_change(self, change: Dict[str, Any]) -> None:
        """Callback when variable dropdown changes."""
        self.plot_data()

    def plot_data(self) -> None:
        """Generate plot based on selected station and variable."""
        with self.output:
            self.output.clear_output(wait=True)
            station: str = self.station_dropdown.value
            variable: str = self.variable_dropdown.value
            if not variable:
                print("No variables available for this station")
                return
            # Get the dataframe
            df: pd.DataFrame = self.cleaned_dfs[station]
            # Check if variable exists in dataframe
            if variable not in df.columns:
                print(f"Variable '{variable}' not found in station {station}")
                return
            # Create plot
            fig, ax = plt.subplots(figsize=(14, 6))
            # Plot the data
            ax.plot(df['Date'], df[variable], linewidth=1.5, color='#2E86AB')
            # Styling
            ax.set_xlabel('Date', fontsize=12, fontweight='bold')
            ax.set_ylabel(variable, fontsize=12, fontweight='bold')
            ax.set_title(
                f'{variable} - Station {station}\n{df["Date"].min().strftime("%d/%m/%Y")} to {df["Date"].max().strftime("%d/%m/%Y")}',
                fontsize=14, fontweight='bold', pad=20
            )
            ax.grid(True, alpha=0.3, linestyle='--')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            # Rotate x-axis labels for better readability
            plt.xticks(rotation=45, ha='right')
            # Add statistics
            stats_text: str = (
                f'Mean: {df[variable].mean():.2f}\n'
                f'Min: {df[variable].min():.2f}\n'
                f'Max: {df[variable].max():.2f}'
            )
            ax.text(
                0.02, 0.98, stats_text, transform=ax.transAxes,
                fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5)
            )
            plt.tight_layout()
            plt.show()

    def display(self) -> None:
        """Display the interactive plot widget."""
        # Display widgets
        display(widgets.VBox([
            widgets.HBox([self.station_dropdown, self.variable_dropdown]),
            self.output
        ]))
        # Generate initial plot
        self.plot_data()

# Extraction/Trasnformation/Visualization of the data (verified code)

### ---Functions---

In [64]:
def data_loading(
    df_name:str,
    path:str = '/content/01.01.2023-01.03.2023/'
) -> pd.DataFrame:
  """
  Data loading function to dynamically load raw weather
  data to pd.DataFrame Object

  Parameters:
  -----------
  df_name: Name/ID of the station
  path: file path

  Returns:
  --------
  df loaded
  """
  file_path = path + df_name
  # Read the header of the file to determine where to
  # trim the columns header
  with open(file_path, 'r', encoding='latin-1') as f:
    for i in range(3):
      line = f.readline()
      if i == 2:
        usecols = len(line.strip().split(','))
  # Pandas csv parser, dynamic
  df = pd.read_csv(
      file_path,
      skiprows=2,
      header=0,
      sep=",",
      decimal=".",        # Symbol to assing decial point
      encoding='latin-1',
      usecols=range(usecols)   # Select the correct columns
  )
  # Skipping first row
  df = df.iloc[1:].copy()
  # Converting each numeric value to numeric type
  for column in df.columns[1:]:
    df[column] = pd.to_numeric(df[column], errors='coerce')

  return df

In [80]:
def standardize_weather_data(
    df,
    station_name,
    variable_map=None
) -> Tuple[pd.DataFrame, Dict[str, List[str]]]:
  """
  Standardize weeather station data by cleaning columns and parsing dates.

  Parameters:
  -----------
  df : Raw dataframe from weather station
  station_name : ID of the station (e.g., 'T0135')
  variable_map : Dictionary to update with station variables.
                 If None, creates new dict.

  Returns:
  --------
  cleaned_df, updated_variable_map
  """
  if variable_map is None:
    variable_map = {}
  # We make a copy to avoid damage
  df_clean = df.copy()
  # Drop "Unnamed" columns
  unnamed_columns = [col for col in df_clean.columns if 'Unnamed' in str(col)]
  df_clean = df_clean.drop(columns=unnamed_columns)
  # Fix the 'Date' column
  df_clean['Date'] = pd.to_datetime(
      df_clean['Date'],
      format='%H:%M:%S %d/%m/%Y'
  )
  # Add Station Identifuier
  df_clean['Station_ID'] = station_name
  # Update Variable map
  variable_cols = [
      col for col in df_clean.columns
      if col not in ['Date', 'Station_ID']
  ]
  variable_map[station_name] = variable_cols
  # Final data
  df_clean = df_clean[['Date'] + ['Station_ID'] + variable_cols]
  return df_clean, variable_map

### ---Sample of Usage---

In [95]:
# Dict comprehension to build python dict for each dfs
dfs = {
  df_name.replace('.csv', ''): data_loading(df_name)
  for df_name in os.listdir('/content/01.01.2023-01.03.2023/')
  if df_name.endswith('.csv')
}
# - cleaned_dfs: dict collection of dfs
# - variable_map: map of the variables for each df
variable_map = {}
cleaned_dfs = {}
for df_name, df in dfs.items():
    cleaned_df, variable_map = standardize_weather_data(df, df_name, variable_map)
    cleaned_dfs[df_name] = cleaned_df
# Plot the result
plotter = WeatherPlotter(cleaned_dfs, variable_map)
plotter.display()